In [1]:
import trino.dbapi
import os 
import polars as pl
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm

import ai4trust_observatory as obs

In [2]:
TRINO_HOST=os.environ.get("TRINO_HOST")
TRINO_USER=os.environ.get("TRINO_USER")

In [ ]:
with trino.dbapi.connect(host=TRINO_HOST, user=TRINO_USER, catalog='iceberg') as conn:
    cur = conn.cursor()
    cur.execute('describe telegram.messages')
    a = cur.description
    schema = cur.fetchall()
    cur.close()
schema

[['date', 'varchar', '', ''],
 ['forwards', 'bigint', '', ''],
 ['message_offset_id', 'bigint', '', ''],
 ['pinned', 'boolean', '', ''],
 ['legacy', 'boolean', '', ''],
 ['query_id', 'varchar', '', ''],
 ['noforwards', 'boolean', '', ''],
 ['data_owner', 'varchar', '', ''],
 ['out', 'boolean', '', ''],
 ['peer_id', 'bigint', '', ''],
 ['from_scheduled', 'boolean', '', ''],
 ['post', 'boolean', '', ''],
 ['edit_hide', 'boolean', '', ''],
 ['nr_replies', 'bigint', '', ''],
 ['id', 'bigint', '', ''],
 ['invert_media', 'boolean', '', ''],
 ['mentioned', 'boolean', '', ''],
 ['_timestamp', 'bigint', '', ''],
 ['text_urls', 'array(varchar)', '', ''],
 ['views', 'bigint', '', ''],
 ['peer_type', 'varchar', '', ''],
 ['silent', 'boolean', '', ''],
 ['_table', 'varchar', '', ''],
 ['media_unread', 'boolean', '', ''],
 ['other_reactions', 'bigint', '', ''],
 ['message', 'varchar', '', ''],
 ['has_comments', 'boolean', '', ''],
 ['query_date', 'varchar', '', ''],
 ['reactions',
  'row(ROLLING_ON_

In [ ]:
eng_cols = ["views"]
cols = ["peer_id", "id", "message", "date", 'media_id', 'media_type'] + eng_cols

views + keyword match

In [24]:
query = "select * from telegram.search_keywords"
keywords= obs.db_utils.df_from_db(query, catalog='db')

In [26]:
keywords.head()

keyword_id,keyword,lang,data_owner,topic
str,str,str,str,str
"""274""","""vaccinations""","""English""","""FBK""","""PUBLIC HEALTH"""
"""275""","""vaccinate""","""English""","""FBK""","""PUBLIC HEALTH"""
"""276""","""vaccinates""","""English""","""FBK""","""PUBLIC HEALTH"""
"""277""","""vaccinated""","""English""","""FBK""","""PUBLIC HEALTH"""
"""278""","""vaccinating""","""English""","""FBK""","""PUBLIC HEALTH"""


In [ ]:
dt_from = dt.date.today() - dt.timedelta(days=7)
dt_to = dt_from + dt.timedelta(days=1)


query = (
        f"select {', '.join(cols)} from telegram.messages"
        f" where date > '{dt_from}' and date < '{dt_to}'"
    )
prio_df = obs.db_utils.df_from_db(query, struct_cols=["reactions"], infer_schema_length=None).filter(pl.col('message').str.contains_any(keywords['keyword'].implode())).with_columns(priority=pl.col('views') / pl.col('views').max())

include measure of channel centrality?

In [ ]:
query = f"select {', '.join(cols)} from telegram.messages limit 1"
df = obs.db_utils.df_from_db(query, struct_cols=["reactions"], infer_schema_length=None)

In [ ]:
channel_id = df['peer_id'].item()

In [ ]:
query = f"select destination as peer_id, count(*) as out_strength from telegram.channels_rels where destination={channel_id} group by destination"
chan_df = obs.db_utils.df_from_db(query, catalog='db')

In [6]:
df

peer_id,id,date,text_urls,other_reactions,views,nr_replies,forwards,reactions
i64,i64,str,list[str],i64,i64,i64,i64,struct[70]
1316425018,3895,"""2025-01-07T12:07:30+00:00""","[""https://elpais.com/espana/2025-01-07/la-extrema-derecha-alcanza-el-16-de-estimacion-de-voto.html""]",0,2196,0,4,"{0,0,0,0,0,0,0,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0}"
